In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, skew

pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.width', 1000)

def perform_EDA(df, filename):
    print(f"=== {filename} EDA Report ===")
    
    # Record counts
    print(f"\nNumber of records: {len(df)}")
    
    # Duplicate Records
    duplicates = len(df) - len(df.drop_duplicates())
    print(f"Number of duplicate records: {duplicates}")
    
    # Dataframe Info
    print(f"\nDataFrame Info:")
    print(df.info())
    
    # Descriptive Stats for Numeric Columns
    print(f"\nDescriptive statistics for numeric columns:")
    print(df.describe())
    
    # Integers
    integer_columns = df.select_dtypes(include='int64').columns.tolist()
    if integer_columns:
        print(f"\nInteger data type columns: {', '.join(integer_columns)}")
    
    # Floats
    float_columns = df.select_dtypes(include='float64').columns.tolist()
    if float_columns:
        print(f"Float data type columns: {', '.join(float_columns)}")
   
    # Objects
    print("\nObject Data Info:")
    object_data = [{
        "Column Name": col, 
        "Number of Unique Values": df[col].nunique(), 
        "Average String Length": round(df[col].str.len().mean(), 2),
        "Max String Length": df[col].str.len().max(),
        "Min String Length": df[col].str.len().min()
    } for col in df.select_dtypes(include='object').columns]
    print(pd.DataFrame(object_data))
    
    # Boolean
    print("\nBoolean Data Info:")
    boolean_data = [{
        "Column Name": col,
        "Number of True Values": df[col].sum(),
        "Number of False Values": len(df) - df[col].sum()
    } for col in df.select_dtypes(include='bool').columns]
    print(pd.DataFrame(boolean_data))
    
    # Rows with Null Values
    null_columns = df.columns[df.isnull().any()].tolist()
    print(f"\nColumns with null values: {', '.join(null_columns)}")
    
    null_rows = df.isnull().any(axis=1).sum()
    print(f"Rows with null values: {null_rows}")

    # Correlation Coefficients between Numeric Columns
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    correlations = {}
    for col in numeric_columns:
        if col != 'totalFare' and df[col].dtype in ['float64', 'int64']:
            correlation, _ = pearsonr(df[col], df['totalFare'])
            correlations[col] = correlation
    print("\nCorrelation Coefficients:")
    print(correlations)

    # High Cardinality
    unique_counts = df.nunique()
    total_records = len(df)
    high_cardinality_cols = unique_counts[unique_counts / total_records > 0.9].index.tolist()
    print("\nHigh Cardinality Columns:")
    print(high_cardinality_cols)
    
    # Skewness
    skewed_cols = {col: skew(df[col].dropna()) for col in numeric_columns}
    print("\nSkewness in Numeric Columns:")
    print(skewed_cols)

    # Visualizations
    dp = sns.displot(df['totalFare'], kde=True, bins=30)
    dp.set_axis_labels("Total Fare", "Frequency")
    plt.title("Distribution of Total Fare")
    plt.show()

    lp = sns.lmplot(x='totalTravelDistance', y='totalFare', data=df)
    lp.set_axis_labels("Total Travel Distance", "Total Fare")
    plt.title("Relationship Between Total Travel Distance and Total Fare")
    plt.show()

    # Correlation Heatmap
    sns.heatmap(df[numeric_columns].corr(), annot=True, cmap="YlGnBu")
    plt.title('Correlation Heatmap of Numeric Features')
    plt.show()

# File paths & names
gcs_bucket = "gs://project-bucket-kl/landing"
filename_list = ['itineraries.csv']

for filename in filename_list:
    print(f"\nWorking on file: {filename}")
    gcs_path = f"{gcs_bucket}/{filename}"
    df = pd.read_csv(gcs_path, sep=',', on_bad_lines='skip', nrows=10000)
    perform_EDA(df, filename)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, to_date
from pyspark.sql.types import IntegerType
import re

gcs_bucket = "gs://project-bucket-kl/landing"
filename = "itineraries.csv"
gcs_path = f"{gcs_bucket}/{filename}"
df = spark.read.csv(gcs_path, header=True, inferSchema=True)

# Display initial record count
initial_count = df.count()
print(f"Total number of records before cleaning: {initial_count}")

# Drop Rows with Null Values in Critical Columns
critical_columns = ['searchDate', 'flightDate', 'startingAirport', 'destinationAirport', 
                    'baseFare', 'totalFare', 'travelDuration', 'totalTravelDistance']
df = df.dropna(subset=critical_columns)

# Drop Unnecessary Columns
drop_columns = ["legId", "fareBasisCode", "segmentsDepartureTimeRaw", 
                "segmentsDepartureTimeEpochSeconds", "segmentsArrivalTimeRaw", 
                "segmentsArrivalTimeEpochSeconds", "segmentsEquipmentDescription", 
                "segmentsDurationInSeconds"]

df = df.drop(*drop_columns)

# Convert date columns to proper date format
date_columns = ["searchDate", "flightDate"]
for date_col in date_columns:
    df = df.withColumn(date_col, to_date(col(date_col), "yyyy-MM-dd"))

# Function to convert duration string to minutes
def duration_to_minutes(duration_str):
    try:
        total_minutes = 0
        if 'H' in duration_str:
            total_minutes += int(re.search(r'(\d+)H', duration_str).group(1)) * 60
        if 'M' in duration_str:
            total_minutes += int(re.search(r'(\d+)M', duration_str).group(1))
        return total_minutes
    except:
        return None  # Returning None will cause the row to be dropped later

to_minutes_udf = udf(duration_to_minutes, IntegerType())

# Convert 'travelDuration' to minutes
df = df.withColumn("travelDurationMinutes", to_minutes_udf("travelDuration"))
df = df.dropna(subset=["travelDurationMinutes"])

# Convert fare and distance columns to double
fare_and_distance_columns = ["baseFare", "totalFare", "totalTravelDistance"]
for col_name in fare_and_distance_columns:
    df = df.withColumn(col_name, col(col_name).cast("double"))

# Final data count after cleaning
final_count = df.count()
print(f"Total number of records after cleaning: {final_count}")

# Save Cleaned Data
output_path = "gs://project-bucket-kl/cleaned/cleaned_data.parquet"
df.write.mode('overwrite').parquet(output_path)
print("Cleaned data saved to Parquet file.")

